# **Data - 나영**

### **Load Modules**

In [1]:
# Utils
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import os
import torchvision.models as models
from utility.early_stopping import EarlyStopping
# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler


import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from tensorboardX import SummaryWriter

# sklearn
from sklearn.metrics import confusion_matrix, classification_report

**Seed Setting**

In [2]:
random.seed(0)

**Device Setting**

In [3]:
# 디바이스 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**Set Hyperparameters**

In [4]:
batch_size = 128
num_epochs = 20
learning_rate = 0.001
momentum = 0.9

# Data Preprocessing

In [5]:
train_val_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(),
    transforms.RandomCrop(32, padding=4), 
    transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])
])

### **Load Data**

**Splitting th training data**

In [6]:
train_val_data = datasets.CIFAR100(root='./data', train=True, download=True, transform=train_val_transform)
test_data = datasets.CIFAR100(root='./data', train=False, download=True, transform=test_transform)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
# train_data = datasets.CIFAR100(root='./data', train=True, download=True, transform=train_transform)
# test_data = datasets.CIFAR100(root='./data', train=False, download=True, transform=test_transform)

In [8]:
# train 데이터를 train/val로 나누기
num_train = len(train_val_data)
indices = list(range(num_train))
split = int(np.floor(0.2 * num_train))  # validation 데이터를 20%로 설정

np.random.shuffle(indices)
train_idx, val_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

**Define DataLoader**

In [9]:
train_loader = DataLoader(train_val_data, batch_size=batch_size, sampler=train_sampler, num_workers=2)
val_loader = DataLoader(train_val_data, batch_size=batch_size, sampler=val_sampler, num_workers=2)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)

# **Model - 하연**

models 폴더에 만들고 import 하는 식으로 해주세요

**Training Loop**

In [10]:
# # models 폴더의 경로 추가
# sys.path.append('./models')

print("use:", device)

# 모델 import 하기
from models.resnetRS import ResNetRS18

# 모델 초기화
net = ResNetRS18()

# 모델을 GPU로 이동
net.to(device)

# 모델 구조 출력
print(summary(net, (3, 224, 224)))

use: cuda:0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
         MaxPool2d-3           [-1, 64, 56, 56]               0
            Conv2d-4           [-1, 64, 56, 56]          36,864
       BatchNorm2d-5           [-1, 64, 56, 56]             128
            Conv2d-6           [-1, 64, 56, 56]          36,864
       BatchNorm2d-7           [-1, 64, 56, 56]             128
        BasicBlock-8           [-1, 64, 56, 56]               0
            Conv2d-9           [-1, 64, 56, 56]          36,864
      BatchNorm2d-10           [-1, 64, 56, 56]             128
           Conv2d-11           [-1, 64, 56, 56]          36,864
      BatchNorm2d-12           [-1, 64, 56, 56]             128
       BasicBlock-13           [-1, 64, 56, 56]               0
           Conv2d-14       

### **Loss and Optimizer**

In [11]:
# 손실함수 초기화
criterion = nn.CrossEntropyLoss()

# 옵티마이저 초기화
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

# 옵티마이저의 state_dict 출력
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61]}]


# **Train - 하연**

### **Model Train**

In [12]:
writer = SummaryWriter("./runs/resnet_18/tensorboard")

In [13]:
# save_path = "./runs/resnet_18/checkpoints"
# early_stopping = EarlyStopping(save_path)
early_stopping = EarlyStopping(patience=5, verbose=True)

In [14]:
# 모델 학습 함수
def train_model(model, trainloader, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for batch_idx, (inputs, labels) in enumerate(trainloader):
            labels = labels.type(torch.LongTensor).to(device)  # CPU에서 long type tensor로 변환
            inputs = inputs.to(device)

            optimizer.zero_grad()

            # 모델 예측
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # 역전파 및 최적화
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

            # 30번째 배치마다 상태 출력
            if (batch_idx + 1) % 30 == 0:
                print(f"Batch [{batch_idx+1}/{len(trainloader)}], Loss: {loss.item():.4f}")

        # Epoch당 평균 손실 계산 및 출력
        epoch_loss = running_loss / len(trainloader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

        # Early Stopping 체크
        early_stopping(epoch_loss, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break

**Model Test**

In [15]:
# 모델 평가 및 테스트 함수 (superclass 예측 포함)
def test_model(model, testloader, criterion, epoch):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)

            # 모델 예측
            outputs = model(inputs)
            test_loss += criterion(outputs, labels).item() * inputs.size(0)
            
            # 예측 결과 저장 및 정확도 계산
            pred = outputs.argmax(dim=1)
            correct += (pred == labels).sum().item()

            # TensorBoard에 테스트 손실 및 정확도 기록
            writer.add_scalar("Test Loss", test_loss / len(testloader.dataset), epoch)
            writer.add_scalar("Test Accuracy", correct / len(testloader.dataset), epoch)

    # 평균 손실 및 정확도 계산
    test_loss /= len(testloader.dataset)
    accuracy = correct / len(testloader.dataset)
    
    return test_loss, accuracy

### **Per-Epoch Activity**

In [ ]:
# Per-Epoch Activity 코드
for epoch in tqdm(range(1, num_epochs + 1)):
    # 모델 학습
    train_model(net, train_loader, criterion, optimizer, num_epochs=num_epochs)
    
    # 테스트 평가
    test_loss, test_accuracy = test_model(net, test_loader, criterion, epoch)
    
    # TensorBoard에 테스트 결과 기록
    writer.add_scalar("Test Loss", test_loss, epoch)
    writer.add_scalar("Test Accuracy", test_accuracy, epoch)

    # 현재 epoch 결과 출력
    print(f"Epoch [{epoch}/{num_epochs}]   Loss: {test_loss:.4f}   Accuracy: {test_accuracy*100:.2f}%")

# TensorBoard writer 닫기
writer.close()

  0%|                                                    | 0/20 [00:00<?, ?it/s]

Batch [30/313], Loss: 5.3023
Batch [60/313], Loss: 5.1915
Batch [90/313], Loss: 4.7582
Batch [120/313], Loss: 4.4300
Batch [150/313], Loss: 4.2861
Batch [180/313], Loss: 4.2456
Batch [210/313], Loss: 4.3841
Batch [240/313], Loss: 4.3330
Batch [270/313], Loss: 4.1105
Batch [300/313], Loss: 4.3609
Epoch [1/20], Loss: 3.7049
Validation loss decreased (inf --> 3.704945).  Saving model ...
Batch [30/313], Loss: 4.0137
Batch [60/313], Loss: 4.0519
Batch [90/313], Loss: 3.9849
Batch [120/313], Loss: 4.0932
Batch [150/313], Loss: 3.9244
Batch [180/313], Loss: 3.7676
Batch [210/313], Loss: 3.8219
Batch [240/313], Loss: 3.8011
Batch [270/313], Loss: 3.8658
Batch [300/313], Loss: 3.9379
Epoch [2/20], Loss: 3.2026
Validation loss decreased (3.704945 --> 3.202595).  Saving model ...
Batch [30/313], Loss: 3.7803
Batch [60/313], Loss: 3.7133
Batch [90/313], Loss: 4.0542
Batch [120/313], Loss: 3.7856
Batch [150/313], Loss: 3.7804
Batch [180/313], Loss: 3.7771
Batch [210/313], Loss: 3.8058
Batch [240/3

  5%|██▏                                        | 1/20 [02:13<42:16, 133.50s/it]

Epoch [1/20]   Loss: 2.4096   Accuracy: 38.69%
Batch [30/313], Loss: 2.1739
Batch [60/313], Loss: 2.1746
Batch [90/313], Loss: 1.9595
Batch [120/313], Loss: 2.2605
Batch [150/313], Loss: 2.0449
Batch [180/313], Loss: 2.1386
Batch [210/313], Loss: 2.4236
Batch [240/313], Loss: 2.1114
Batch [270/313], Loss: 2.5650
Batch [300/313], Loss: 2.2065
Epoch [1/20], Loss: 1.8136
Validation loss decreased (1.844894 --> 1.813579).  Saving model ...
Batch [30/313], Loss: 2.1287
Batch [60/313], Loss: 2.2129
Batch [90/313], Loss: 2.1800
Batch [120/313], Loss: 2.0237
Batch [150/313], Loss: 2.5200
Batch [180/313], Loss: 2.2684
Batch [210/313], Loss: 2.0880
Batch [240/313], Loss: 2.1274
Batch [270/313], Loss: 2.1275
Batch [300/313], Loss: 2.0116
Epoch [2/20], Loss: 1.7739
Validation loss decreased (1.813579 --> 1.773912).  Saving model ...
Batch [30/313], Loss: 2.1107
Batch [60/313], Loss: 2.3633
Batch [90/313], Loss: 1.9269
Batch [120/313], Loss: 1.9945
Batch [150/313], Loss: 2.0662
Batch [180/313], Los

 10%|████▎                                      | 2/20 [04:25<39:51, 132.88s/it]

Epoch [2/20]   Loss: 2.2295   Accuracy: 44.74%
Batch [30/313], Loss: 1.3519
Batch [60/313], Loss: 1.2732
Batch [90/313], Loss: 1.5231
Batch [120/313], Loss: 1.5140
Batch [150/313], Loss: 1.6289
Batch [180/313], Loss: 1.3187
Batch [210/313], Loss: 1.5221
Batch [240/313], Loss: 1.6114
Batch [270/313], Loss: 1.5023
Batch [300/313], Loss: 1.5804
Epoch [1/20], Loss: 1.2062
Validation loss decreased (1.228343 --> 1.206229).  Saving model ...
Batch [30/313], Loss: 1.4971
Batch [60/313], Loss: 1.3450
Batch [90/313], Loss: 1.3157
Batch [120/313], Loss: 1.5741
Batch [150/313], Loss: 1.4132
Batch [180/313], Loss: 1.5135
Batch [210/313], Loss: 1.2469
Batch [240/313], Loss: 1.3625
Batch [270/313], Loss: 1.5136
Batch [300/313], Loss: 1.6751
Epoch [2/20], Loss: 1.1685
Validation loss decreased (1.206229 --> 1.168496).  Saving model ...
Batch [30/313], Loss: 1.4670
Batch [60/313], Loss: 1.3587
Batch [90/313], Loss: 1.4445
Batch [120/313], Loss: 1.5624
Batch [150/313], Loss: 1.7404
Batch [180/313], Los

 15%|██████▍                                    | 3/20 [06:38<37:36, 132.74s/it]

Epoch [3/20]   Loss: 2.4292   Accuracy: 45.09%
Batch [30/313], Loss: 0.8907
Batch [60/313], Loss: 0.8085
Batch [90/313], Loss: 1.0777
Batch [120/313], Loss: 1.0519
Batch [150/313], Loss: 0.9564
Batch [180/313], Loss: 1.0441
Batch [210/313], Loss: 0.9786
Batch [240/313], Loss: 0.8696
Batch [270/313], Loss: 1.0113
Batch [300/313], Loss: 0.9897
Epoch [1/20], Loss: 0.7339
Validation loss decreased (0.748711 --> 0.733936).  Saving model ...
Batch [30/313], Loss: 0.9393
Batch [60/313], Loss: 0.8441
Batch [90/313], Loss: 0.9259
Batch [120/313], Loss: 0.6647
Batch [150/313], Loss: 1.0327
Batch [180/313], Loss: 1.0460
Batch [210/313], Loss: 1.1740
Batch [240/313], Loss: 0.7867
Batch [270/313], Loss: 0.8857
Batch [300/313], Loss: 0.8685
Epoch [2/20], Loss: 0.7084
Validation loss decreased (0.733936 --> 0.708357).  Saving model ...
Batch [30/313], Loss: 0.8308
Batch [60/313], Loss: 0.8094
Batch [90/313], Loss: 0.9613
Batch [120/313], Loss: 0.7402
Batch [150/313], Loss: 1.1394
Batch [180/313], Los

 20%|████████▌                                  | 4/20 [08:52<35:30, 133.13s/it]

Epoch [4/20]   Loss: 2.7058   Accuracy: 45.19%
Batch [30/313], Loss: 0.3761
Batch [60/313], Loss: 0.4670
Batch [90/313], Loss: 0.4061
Batch [120/313], Loss: 0.6016
Batch [150/313], Loss: 0.6680
Batch [180/313], Loss: 0.6241
Batch [210/313], Loss: 0.5216
Batch [240/313], Loss: 0.6144
Batch [270/313], Loss: 0.4618
Batch [300/313], Loss: 0.4761
Epoch [1/20], Loss: 0.4070
Validation loss decreased (0.416428 --> 0.407026).  Saving model ...
Batch [30/313], Loss: 0.4127
Batch [60/313], Loss: 0.4773
Batch [90/313], Loss: 0.4568
Batch [120/313], Loss: 0.7312
Batch [150/313], Loss: 0.5654
Batch [180/313], Loss: 0.4435
Batch [210/313], Loss: 0.4968
Batch [240/313], Loss: 0.4967
Batch [270/313], Loss: 0.5701
Batch [300/313], Loss: 0.4787
Epoch [2/20], Loss: 0.3968
Validation loss decreased (0.407026 --> 0.396781).  Saving model ...
Batch [30/313], Loss: 0.5334
Batch [60/313], Loss: 0.5181
Batch [90/313], Loss: 0.5376
Batch [120/313], Loss: 0.4445
Batch [150/313], Loss: 0.4975
Batch [180/313], Los

### **Result**

In [ ]:
print(f" Result of ResNet = Epoch : {epoch}   Loss : {test_loss}   Accuracy : {test_accuracy}")

# Test - 나영(Accuracy) 현욱(Analysis)

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('10000개 테스트 이미지에서 모델 정확도: %d %%' % (
    100 * correct / total))

**Visualization of average loss(수정 필요)**

In [ ]:
train_losses = []
test_losses = []
test_accuracies = []

for epoch in range(num_epochs):
    # 훈련과정에서 손실을 기록
    train_loss = 0
    total_samples = 0
    net.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
        total_samples += inputs.size(0)
    
    train_losses.append(train_loss / total_samples)

    # 평가 과정에서 손실과 정확도를 기록
    test_loss = 0
    correct = 0
    total = 0
    net.eval()
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    test_losses.append(test_loss / total)
    test_accuracies.append(correct / total)

# 손실과 정확도 그래프 그리기
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(test_accuracies, label='Test Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

**Top-1 Accuracy**

In [ ]:
def calculate_top1_accuracy(model, device, data_loader, criterion):
    model.eval()  # 모델을 평가 모드로 설정
    correct = 0
    total = 0
    
    with torch.no_grad():  # 그래디언트 계산 비활성화
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

            # # 각 샘플에 대한 예측 결과와 실제 레이블 출력
            # for i in range(data.size(0)):
            #     print(f"Sample {i + 1}: Predicted = {predicted[i].item()}, Actual = {target[i].item()}")

    top1_accuracy = 100 * correct / total
    print(f"Top-1 Accuracy: {top1_accuracy:.2f}%")

In [ ]:
# 모델 훈련 후 검증 데이터셋에 대한 Top-1 정확도 계산 및 출력
calculate_top1_accuracy(net, device, val_loader, criterion)

**Top-5 Accuracy**

In [ ]:
def calculate_and_print_top5_accuracy(model, device, data_loader, criterion):
    model.eval()  # 모델을 평가 모드로 설정
    correct = 0
    total = 0
    
    with torch.no_grad():  # 그래디언트 계산 비활성화
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            # Top-5 예측 결과 가져오기
            _, predicted_top5 = torch.topk(outputs, 5, dim=1)
            total += target.size(0)
            
            # 예측된 Top-5 내에 실제 레이블이 있는지 확인
            correct += (predicted_top5 == target.view(-1, 1)).sum().item()

    top5_accuracy = 100 * correct / total
    print(f"Top-5 Accuracy: {top5_accuracy:.2f}%")

In [ ]:
calculate_and_print_top5_accuracy(net, device, val_loader, criterion)

# Data Analysis - 현욱

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir=./runs/resnet_18/tensorboard --port=8202 --host=0.0.0.0

### **Classes**

In [ ]:
classes = train_data.classes
coarse_classes = [
    'aquatic mammals', 'fish', 'flowers', 'food containers', 'fruit and vegetables', 'household electrical devices', 
    'household furniture', 'insects', 'large carnivores', 'large man-made outdoor things', 
    'large natural outdoor scenes', 'large omnivores and herbivores', 'medium-sized mammals', 
    'non-insect invertebrates', 'people', 'reptiles', 'small mammals', 'trees', 'vehicles 1', 'vehicles 2'
]

In [ ]:
print(len(classes), len(coarse_classes))

##### **Fine_to_coarse_mapping**

In [ ]:
# CIFAR-100 세부 클래스(fine classes)와 상위 클래스(coarse classes) 매핑
fine_to_coarse_mapping = {
    # aquatic mammals
    'beaver': 'aquatic mammals',
    'dolphin': 'aquatic mammals',
    'otter': 'aquatic mammals',
    'seal': 'aquatic mammals',
    'whale': 'aquatic mammals',
    
    # fish
    'aquarium fish': 'fish',
    'flatfish': 'fish',
    'ray': 'fish',
    'shark': 'fish',
    'trout': 'fish',
    
    # flowers
    'orchids': 'flowers',
    'poppies': 'flowers',
    'roses': 'flowers',
    'sunflowers': 'flowers',
    'tulips': 'flowers',
    
    # food containers
    'bottles': 'food containers',
    'bowls': 'food containers',
    'cans': 'food containers',
    'cups': 'food containers',
    'plates': 'food containers',
    
    # fruit and vegetables
    'apples': 'fruit and vegetables',
    'mushrooms': 'fruit and vegetables',
    'oranges': 'fruit and vegetables',
    'pears': 'fruit and vegetables',
    'sweet peppers': 'fruit and vegetables',
    
    # household electrical devices
    'clock': 'household electrical devices',
    'computer keyboard': 'household electrical devices',
    'lamp': 'household electrical devices',
    'telephone': 'household electrical devices',
    'television': 'household electrical devices',
    
    # household furniture
    'bed': 'household furniture',
    'chair': 'household furniture',
    'couch': 'household furniture',
    'table': 'household furniture',
    'wardrobe': 'household furniture',
    
    # insects
    'bee': 'insects',
    'beetle': 'insects',
    'butterfly': 'insects',
    'caterpillar': 'insects',
    'cockroach': 'insects',
    
    # large carnivores
    'bear': 'large carnivores',
    'leopard': 'large carnivores',
    'lion': 'large carnivores',
    'tiger': 'large carnivores',
    'wolf': 'large carnivores',
    
    # large man-made outdoor things
    'bridge': 'large man-made outdoor things',
    'castle': 'large man-made outdoor things',
    'house': 'large man-made outdoor things',
    'road': 'large man-made outdoor things',
    'skyscraper': 'large man-made outdoor things',
    
    # large natural outdoor scenes
    'cloud': 'large natural outdoor scenes',
    'forest': 'large natural outdoor scenes',
    'mountain': 'large natural outdoor scenes',
    'plain': 'large natural outdoor scenes',
    'sea': 'large natural outdoor scenes',
    
    # large omnivores and herbivores
    'camel': 'large omnivores and herbivores',
    'cattle': 'large omnivores and herbivores',
    'chimpanzee': 'large omnivores and herbivores',
    'elephant': 'large omnivores and herbivores',
    'kangaroo': 'large omnivores and herbivores',
    
    # medium-sized mammals
    'fox': 'medium-sized mammals',
    'porcupine': 'medium-sized mammals',
    'possum': 'medium-sized mammals',
    'raccoon': 'medium-sized mammals',
    'skunk': 'medium-sized mammals',
    
    # non-insect invertebrates
    'crab': 'non-insect invertebrates',
    'lobster': 'non-insect invertebrates',
    'snail': 'non-insect invertebrates',
    'spider': 'non-insect invertebrates',
    'worm': 'non-insect invertebrates',
    
    # people
    'baby': 'people',
    'boy': 'people',
    'girl': 'people',
    'man': 'people',
    'woman': 'people',
    
    # reptiles
    'crocodile': 'reptiles',
    'dinosaur': 'reptiles',
    'lizard': 'reptiles',
    'snake': 'reptiles',
    'turtle': 'reptiles',
    
    # small mammals
    'hamster': 'small mammals',
    'mouse': 'small mammals',
    'rabbit': 'small mammals',
    'shrew': 'small mammals',
    'squirrel': 'small mammals',
    
    # trees
    'maple': 'trees',
    'oak': 'trees',
    'palm': 'trees',
    'pine': 'trees',
    'willow': 'trees',
    
    # vehicles 1
    'bicycle': 'vehicles 1',
    'bus': 'vehicles 1',
    'motorcycle': 'vehicles 1',
    'pickup truck': 'vehicles 1',
    'train': 'vehicles 1',
    
    # vehicles 2
    'lawn-mower': 'vehicles 2',
    'rocket': 'vehicles 2',
    'streetcar': 'vehicles 2',
    'tank': 'vehicles 2',
    'tractor': 'vehicles 2'
}


### **Confusion Matrix**

In [ ]:
y_pred = []
y_true = []

# iterate over test data
for x, y in torch.utils.data.DataLoader(dataset=test_data, batch_size=batch_size):
    
    #print('iter val', i)
    x = x.to(device)
    y = y.to(device)
    z = net(x)
    _, yhat = torch.max(z, 1)
    pred = yhat.data.cpu().numpy()
    y_pred.extend(pred) # Save Prediction

    labels = y.data.cpu().numpy()
    y_true.extend(labels) # Save Truth

# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix) *10, index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (128,70))
sns.heatmap(df_cm, annot=True)
plt.title('Confusion Matrix of ResNet (CIFAR100)')
plt.savefig('./runs/resnet_18/Confusion_matrix_ResNet_Cifar100.jpg')

### **Confusion Matrix - Coarse**

### **Classification Report**

In [ ]:
print(f" Classification Report of ResNet(CIFAR100)  \n { classification_report(y_true, y_pred)}")

In [ ]:
# Save Classification Report as txt file 
with open("./runs/resnet_18/cr_ResNet18.txt", "w") as text_file:
    print(classification_report(y_true, y_pred, digits=4), file=text_file)

In [ ]:
# Load Classification Report txt file 
with open("./runs/resnet_18/cr_ResNet18.txt", "r") as f:
  cr = f.read()
print(cr)